In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, AveragePooling2D
from keras.applications import MobileNetV2
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import class_weight

In [83]:
# import keras.applications.resnet50.ResNet101V2
base_model=keras.applications.vgg19.VGG19(weights='imagenet',include_top=False, input_shape=(299,299, 3))

80142336/80134624 [==============================] - 7s 0us/step


In [0]:
dir(keras.applications)

In [0]:
x=base_model.output
x=AveragePooling2D(pool_size=(7, 7), strides=None)(x)
x=GlobalAveragePooling2D()(x)
x=Dense(256,activation='relu')(x)
x=Dense(128,activation='relu')(x)
x=Dense(64,activation='relu')(x)
preds=Dense(5,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)

for layer in base_model.layers:
    layer.trainable=False

In [85]:
train_datagen=ImageDataGenerator(shear_range=0.2, horizontal_flip=True, zoom_range=0.2)

train_generator=train_datagen.flow_from_directory('gdrive/My Drive/flower_photos',
                                                 target_size=(299,299),
                                                 color_mode='rgb',
                                                 batch_size=4,
                                                 class_mode='categorical',
                                                 shuffle=True)


Found 3670 images belonging to 5 classes.


In [0]:
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)

In [87]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10, class_weight=class_weights)

Epoch 1/10
917/917 [==============================] - 201s 220ms/step - loss: 0.7225 - acc: 0.7555
Epoch 2/10
917/917 [==============================] - 172s 187ms/step - loss: 0.4556 - acc: 0.8389
Epoch 3/10
917/917 [==============================] - 171s 187ms/step - loss: 0.3883 - acc: 0.8680
Epoch 4/10
917/917 [==============================] - 171s 187ms/step - loss: 0.3418 - acc: 0.8811
Epoch 5/10
917/917 [==============================] - 171s 187ms/step - loss: 0.3002 - acc: 0.8978
Epoch 6/10
917/917 [==============================] - 171s 187ms/step - loss: 0.2872 - acc: 0.9016
Epoch 7/10
917/917 [==============================] - 172s 187ms/step - loss: 0.2579 - acc: 0.9081
Epoch 8/10
917/917 [==============================] - 172s 187ms/step - loss: 0.2303 - acc: 0.9201
Epoch 9/10
917/917 [==============================] - 171s 187ms/step - loss: 0.2236 - acc: 0.9242
Epoch 10/10
917/917 [==============================] - 171s 187ms/step - loss: 0.2029 - acc: 0.9313


In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con